In [1]:
!git clone https://github.com/congnghia0609/ntc-scv.git

Cloning into 'ntc-scv'...
Updating files:  72% (8/11)
Updating files:  81% (9/11)
Updating files:  90% (10/11)
Updating files: 100% (11/11)
Updating files: 100% (11/11), done.


In [3]:
import zipfile
import os

train_path = "./ntc-scv/data/data_train.zip"
test_path = "./ntc-scv/data/data_test.zip"
extract_path = "./data/ntc-scv"



# Ensure the extraction folder exists
os.makedirs(extract_path, exist_ok=True)

# Unzipping the file
with zipfile.ZipFile(train_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

with zipfile.ZipFile(test_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")

Files extracted to: ./data/ntc-scv


In [5]:
import os
import pandas as pd

def load_data_from_path(folder_path):
    examples = []
    for label in os.listdir(folder_path):
        full_path = os.path.join(folder_path, label)
        for file_name in os.listdir(full_path):
            file_path = os.path.join(full_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            sentence = " ".join(lines)
            if label == "neg":
                label = 0
            if label == "pos":
                label = 1
            data = {
                'sentence': sentence,
                'label': label
            }
            examples.append(data)
    return pd.DataFrame(examples)

In [8]:
folder_paths = {
    'train': './data/ntc-scv/data_train/train',
    'valid': './data/ntc-scv/data_train/test',
    'test': './data/ntc-scv/data_test/test'
}

train_df = load_data_from_path(folder_paths['train'])
valid_df = load_data_from_path(folder_paths['valid'])
test_df = load_data_from_path(folder_paths['test'])

In [9]:
!pip install langid

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------- ----------------------------- 0.5/1.9 MB 4.2 MB/s eta 0:00:01
     ---------------------------------------- 1.9/1.9 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941178 sha256=fee57c8f6255e37447dcb71fcff00808d979404654eb44de613fee7c6b871f57
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\23\c8\c6\eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [10]:
from langid.langid import LanguageIdentifier, model


def identify_vn(df):
    identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
    not_vi_idx = set()
    THRESHOLD = 0.9
    for idx, row in df.iterrows():
        score = identifier.classify(row["sentence"])
        if score[0] != "vi" or (score[0] == "vi" and score[1] <= THRESHOLD):
            not_vi_idx.add(idx)
    vi_df = df[~df.index.isin(not_vi_idx)]
    not_vi_df = df[df.index.isin(not_vi_idx)]
    return vi_df, not_vi_df